# Image Vectorization using Traditional method

In [4]:
!pip install opencv-python
!pip install numpy
!pip install scipy
!pip install svgwrite

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 640.0 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 406.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.6 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.2/38.6 MB 1.5 MB/s eta 0:00:25
   ---------------------------------------- 0.2/38.6 MB 1.5 MB/s eta 0:00:25
   ---------------------------------------- 0.3/38.6 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.6/38.6 MB 2.3 MB/s eta 0:00:17
    --------------------------------------- 0.9/38.6 MB 2.9 MB/s eta 0:00:13
   - -------------------------------------- 1.1/38.6 MB 3.1 MB/s eta 0:00:13
   - -------------------------------------- 1.2/38.6 MB 2.8 MB/s eta 0:00:14
  

In [8]:
# import packages
import cv2
import numpy as np

# step 1: Interpolation
use bilinear interpolation

In [16]:
# Read the image
image = cv2.imread('src/2.jpg', cv2.IMREAD_GRAYSCALE)
height, width = image.shape

# Convert RGB to Lab
# lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

# Split the channels
# L, a, b = cv2.split(lab_image)

# Resize each channel using bilinear interpolation
resized_image = cv2.resize(image, (width * 5, height * 5), interpolation=cv2.INTER_LINEAR)
# L_resized = cv2.resize(L, (width * 5, height * 5), interpolation=cv2.INTER_LINEAR)
# a_resized = cv2.resize(a, (width * 5, height * 5), interpolation=cv2.INTER_LINEAR)
# b_resized = cv2.resize(b, (width * 5, height * 5), interpolation=cv2.INTER_LINEAR)

# Merge the channels back
# resized_lab_image = cv2.merge((L_resized, a_resized, b_resized))

# Convert back to RGB
# resized_image = cv2.cvtColor(resized_lab_image, cv2.COLOR_Lab2BGR)

# Save or display the resized image
cv2.imwrite('output/resized_image.png', resized_image)

# Set a threshold value
threshold_value = 220

# Apply the threshold
_, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
_, thresholded_image_with_resize = cv2.threshold(resized_image, threshold_value, 255, cv2.THRESH_BINARY)

# invert the image
inverted_image = 255 - thresholded_image
inverted_image_with_resize = 255 - thresholded_image_with_resize

# Find contours in the binary image
contours, _ = cv2.findContours(inverted_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours_with_resize, _ = cv2.findContours(inverted_image_with_resize, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Create an empty image to draw contours on (make sure it's a 3-channel image)
contour_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
contour_image_with_resize = np.zeros((resized_image.shape[0], resized_image.shape[1], 3), dtype=np.uint8)

# Draw the contours on the empty image
# Using a visible color (e.g., green) and a suitable thickness
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 1)  # Green color, thickness of 1
cv2.drawContours(contour_image_with_resize, contours_with_resize, -1, (0, 255, 0), 1)  # Green color, thickness of 1

# Save or display the contour image
cv2.imwrite('output/threshold_image.png', thresholded_image)
cv2.imwrite('output/threshold_image_with_resize.png', thresholded_image_with_resize)
cv2.imwrite('output/contour_image.png', contour_image)
cv2.imwrite('output/contour_image_with_resize.png', contour_image_with_resize)



True

# step 2: use contour points to generate SVG file

In [15]:
import cv2
import numpy as np
from scipy.interpolate import splprep, splev
import svgwrite

# Step 1: Extract Contour Points
# image = cv2.imread('src/2.jpg', cv2.IMREAD_GRAYSCALE)
# _, thresh = cv2.threshold(image, 220, 255, cv2.THRESH_BINARY)
# contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_with_resize = max(contours_with_resize, key=len)
points = contour_with_resize[:, 0, :]  # Remove unnecessary dimension
x, y = points[:, 0], points[:, 1]

# Step 2: Generate Bezier Curves
tck, u = splprep([x, y], s=0, k=3)
u_new = np.linspace(u.min(), u.max(), num=len(points)*10)
x_new, y_new = splev(u_new, tck)

# Step 3: Create SVG File
dwg = svgwrite.Drawing('output/2.svg', profile='tiny')
path_data = f'M {x_new[0]},{y_new[0]} '

for i in range(1, len(x_new)-1, 3):
    path_data += f'C {x_new[i]},{y_new[i]} {x_new[i+1]},{y_new[i+1]} {x_new[i+2]},{y_new[i+2]} '

dwg.add(dwg.path(d=path_data, stroke='black', fill='none'))
dwg.save()


In [19]:
import cv2
import numpy as np
from scipy.interpolate import splprep, splev
import svgwrite

# Step 1: Extract Contour Points
# image = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE)
# _, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
# contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Select the largest contour
contour = max(contours, key=len)

# Step 2: Fit Bezier Curves
# Convert contour points to a format suitable for splprep
points = contour[:, 0, :]  # Remove unnecessary dimension
x, y = points[:, 0], points[:, 1]

# Fit spline to contour points
tck, u = splprep([x, y], s=0, k=3)

# Evaluate spline
u_new = np.linspace(u.min(), u.max(), num=len(points)*10)
x_new, y_new = splev(u_new, tck)

# Step 3: Create SVG File
dwg = svgwrite.Drawing('output/2_with_resize.svg', profile='tiny')

# Move to the start point
path_data = f'M {x_new[0]},{y_new[0]} '

# Create path data for cubic Bezier curve
for i in range(1, len(x_new)-1, 3):
    if i + 2 < len(x_new):
        path_data += f'C {x_new[i]},{y_new[i]} {x_new[i+1]},{y_new[i+1]} {x_new[i+2]},{y_new[i+2]} '

# Add the path to the SVG
dwg.add(dwg.path(d=path_data, stroke='black', fill='none'))

# Save the SVG file
dwg.save()
